In [90]:
#%esptool erase
## esptool.py --chip esp32 --port /dev/ttyACM0 erase_flash
#%esptool esp32 /home/julian/executables-impure/espbinaries/esp32-20220413-unstable-v1.18-330-gd242a9b7f.bin
## esptool.py --chip esp32 --port /dev/ttyUSB1 --baud 460800 write_flash -z 0x1000 executables-impure/espbinaries/esp32-20210902-v1.17.bin 
%serialconnect

# when running:
# mosquitto_sub -h mosquitto.doesliverpool.xyz -v -t "esp32/AB4634/#"



***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fafc00e5730, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to --port=/dev/ttyUSB2 --baud=115200 
Ready.


In [68]:
%sendtofile config.txt

connection3  HUAWEI-B535-2F99,EYR8QJE65GT,mosquitto.doesliverpool.xyz
connection2  JulianS9,beckaaaa,mosquitto.doesliverpool.xyz
connection0  DoESLiverpool,decafbad00,mosquitto.doesliverpool.xyz
connection1  BT-STCPTN,FNPmKv4V7hMyJy,mosquitto.doesliverpool.xyz
pinled       2
pinscl       13
pinsda       12
uartbus      2
wdtsecs      300
pinfan       25
position     tupperware

Sent 11 lines (379 bytes) to config.txt.


In [25]:
%sendtofile --source stdmqttas.py
%sendtofile --quiet --source utils.py

Sent 82 lines (2679 bytes) to stdmqttas.py.
Sent 32 lines (969 bytes) to utils.py.


In [26]:
# This requires you to checkout https://github.com/peterhinch/micropython-mqtt/tree/master/mqtt_as
%sendtofile --source /home/julian/extrepositories/micropython-mqtt/mqtt_as/mqtt_as.py

# !!! The redo for different connections isn't working so put the one 
# that is active into connection0

Sent 647 lines (23171 bytes) to mqtt_as.py.


In [85]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB0 --baud=115200 
Traceback (most recent call last):
  File "main.py", line 3, in <module>
  File "stdmqttas.py", line 3, in <module>
  File "mqtt_as.py", line 35, in <module>
KeyboardInterrupt: 
MicroPython v1.17 on 2021-09-02; ESP32 module with ESP32
Type "help()" for more information.
>>> 
>>> 
raw REPL; CTRL-B to exit
>Ready.


In [86]:
%sendtofile main.py

import time, sys, machine, struct
import uasyncio as asyncio
from stdmqttas import fconfig, config, mqttconnecttask, callbackcmdtask
from stdmqttas import shortmac, topicstem, itertools_count
from stdmqttas import pinled, pinledOnvalue, flashpinled
from mqtt_as import MQTTClient

import network
network.WLAN().active(0)  # disable the connection at startup
flashpinled(5, 300, 300)

def crc8(b):
    crc = 0xFF
    for c in b:
        crc ^= c
        for i in range(8):
            crc = 0xFF & (((crc << 1) ^ 0x31) if (crc & 0x80) else (crc << 1))
    return crc

i2c = machine.SoftI2C(scl=machine.Pin(int(fconfig["pinscl"])), sda=machine.Pin(int(fconfig["pinsda"])))
uart = machine.UART(int(fconfig["uartbus"]), baudrate=9600)

topicscd30 = topicstem+"/scd30"
topiccdm = topicstem+"/CDM7160"
topicmhz = topicstem+"/MHZ19"
topiccmd = topicstem+"/cmd"
topicreply = topicstem+"/reply"
topiconline = topicstem+"/online"
topicip = topicstem+"/ip"
topicerror = topicstem+"/error"
sensorposition = fconfig.get("position", "")
print("topicstem: '%s'" % topicstem)
print("sensorposition: '%s'" % sensorposition)
pinfan = machine.Pin(int(fconfig.get("pinfan", 25)), machine.Pin.OUT)
pinfan.value(0)

async def WatchDogtask(client, wdtsecs):
    await asyncio.sleep_ms(20*1000)
    print("Starting Watchdog timer for", wdtsecs, "seconds")
    wd = machine.WDT(timeout=int(wdtsecs*1000))
    while True:
        await asyncio.sleep_ms(1*1000)
        if client.isconnected():
            pinfan.value(1)
            wd.feed()
            pinled.value(pinledOnvalue)
            await asyncio.sleep_ms(50)
            pinled.value(not pinledOnvalue)
            await asyncio.sleep_ms(9*1000)
        elif pinled:
            pinfan.value(0)
            pinled.value(pinledOnvalue)
            await asyncio.sleep_ms(1*1000)
            pinled.value(not pinledOnvalue)
            print("Not feeding watchdog")
    
async def CDM7160_task(client, i2c):
    try:
        while True:
            try:
                k = i2c.readfrom_mem(0x69, 0x03, 2)
                v = (k[0]+k[1]*256)
            except OSError:
                v = 0
            if v != 0:
                print(topiccdm, v)
                if client.isconnected():
                    await client.publish(topiccdm, "%f %s"%(v, sensorposition))
                    pinled.value(pinledOnvalue)
                    await asyncio.sleep_ms(50)
                    pinled.value(not pinledOnvalue)
                await asyncio.sleep_ms(3000)
            else:
                await asyncio.sleep_ms(1000)
    except OSError as e:
        print("CDM7160taskerror", e)
    await asyncio.sleep_ms(5000)
    aloop.create_task(CDM7160_task(client, i2c))
    
async def MHZ19_task(client, uart):
    try:
        uart.write(bytes([0xFF, 0x01, 0x79, 0x00, 0x00, 0x00, 0x00, 0x00, 0x86]))
        while True:
            await asyncio.sleep_ms(2000)
            uart.write(bytes([0xFF, 0x01, 0x86, 0x00, 0x00, 0x00, 0x00, 0x00, 0x79]))
            await asyncio.sleep_ms(100)
            s = uart.read()
            if s and len(s) == 9 and s[0] == 0xFF and s[1] == 0x86:
                v = s[2]*256 + s[3]
                print(topicmhz, v)
                if client.isconnected():
                    await client.publish(topicmhz, "%d %s"%(v, sensorposition))
                    pinled.value(pinledOnvalue)
                    await asyncio.sleep_ms(50)
                    pinled.value(not pinledOnvalue)
    except OSError as e:
        print("MHZ19_task", e)
    await asyncio.sleep_ms(5000)
    aloop.create_task(MHZ19_task(client, uart))

    
async def SCD30task(client, i2c, topicscd30):
    try:
        i2c.writeto(0x61, bytes([0x02, 0x04]))   # soft reset
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x00, 0x10, 0x00, 0x00, crc8([0x00, 0x00])]))   # continuous measurement
        await asyncio.sleep_ms(40)
        i2c.writeto(0x61, bytes([0x46, 0x00, 0x00, 0x02, crc8([0x00, 0x02])]))   # 2s measurement interval
        await asyncio.sleep_ms(40)
        for n in itertools_count():
            y = i2c.writeto(0x61, bytes([0x02, 0x02]))  # data ready?
            await asyncio.sleep_ms(4)
            d = i2c.readfrom(0x61, 2)    # ready is 1
            if d == bytes([0x00, 0x01]):
                y = i2c.writeto(0x61, bytes([0x03, 0x00]))  # read data
                await asyncio.sleep_ms(4)
                x = i2c.readfrom(0x61, 18)
                crcvalid = (len(x) == 18) and (crc8(x[0:2]) == x[2]) and (crc8(x[3:5]) == x[5]) and (crc8(x[6:8]) == x[8]) and (crc8(x[9:11]) == x[11]) and (crc8(x[12:14]) == x[14]) and (crc8(x[15:17]) == x[17])
                co2, temp, humid = struct.unpack(">fff", bytes([x[0],x[1],x[3],x[4], x[6],x[7],x[9],x[10], x[12],x[13],x[15],x[16]]))
                print(n, topicscd30, co2, temp, humid, crcvalid)
                if client.isconnected() and crcvalid:
                    await client.publish(topicscd30, "%f %f %f %s"%(co2, temp, humid, sensorposition))
                    pinled.value(pinledOnvalue)
                    await asyncio.sleep_ms(50)
                    pinled.value(not pinledOnvalue)
            await asyncio.sleep_ms(500)
    except OSError as e:
        print("SCD30taskerror", topicscd30, e)
    await asyncio.sleep_ms(5000)
    aloop.create_task(SCD30task(client, i2c, topicscd30))

def callbackcmd(topic, msg, retained):
    aloop.create_task(callbackcmdtask(client, topicreply, msg))

async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.publish(topiconline, "1", retain=True)
    await client.publish(topicip, ipnumber, retain=True)
    await client.subscribe(topiccmd, 1)

config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
config['will'] = (topiconline, "0", True)
client = MQTTClient(config)
client.DEBUG = True

aloop = asyncio.get_event_loop()
aloop.create_task(SCD30task(client, i2c, topicscd30))
aloop.create_task(CDM7160_task(client, i2c))
aloop.create_task(MHZ19_task(client, uart))
aloop.create_task(mqttconnecttask(client))
if "wdtsecs" in fconfig:
    aloop.create_task(WatchDogtask(client, float(fconfig["wdtsecs"])))

print("first clearing topicerror")
aloop.run_until_complete(client.publish(topicerror, "", retain=True))
for n in itertools_count():
    print("runforever", n)
    try:
        aloop.run_forever()
    except Exception as e:
        aloop.run_until_complete(client.publish(topicerror, str(n)+" "+str(e), retain=True))


Sent 163 lines (6515 bytes) to main.py.


In [80]:
%serialconnect

import machine
uart = machine.UART(2, baudrate=9600)



serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [62]:
import time

uart.write(bytes([0xFF, 0x01, 0x79, 0x00, 0x00, 0x00, 0x00, 0x00, 0x86]))
for i in range(10):
uart.write(bytes([0xFF, 0x01, 0x86, 0x00, 0x00, 0x00, 0x00, 0x00, 0x79]))
time.sleep_ms(100)
k = uart.read()
print(k)


b'\xffy\x01\x00\x00\x00\x00\x00\x86\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'
b'\xff\x86\x01\xedA\x00\x00\x00K'


In [64]:
print(len(k), k[2]*256 + k[3])

9 493


In [13]:
b = bytes([0xFF, 0x01, 0x86, 0x00, 0x00, 0x00, 0x00, 0x00, 0x79])
print(b)

b'\xff\x01\x86\x00\x00\x00\x00\x00y'
